In [ ]:
pip install transformers

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

# Load the tokenizer and pre-trained model
tokenizer = AutoTokenizer.from_pretrained("salesforce/codegen-350M-multi")
model = AutoModelForCausalLM.from_pretrained("salesforce/codegen-350M-multi")

In [ ]:
import os
directory = "/content/connectors"
text_files = []

# Convert .go files in the "connector" directory to text files
for file in os.listdir(directory):
    if file.endswith(".go"):
        file_path = os.path.join(directory, file)
        with open(file_path, "r") as f:
            text = f.read()
            text_files.append(text)
print(len(text_files))          

3


In [ ]:
train_size = int(0.8 * len(text_files))
train_texts = text_files[:train_size]
val_texts = text_files[train_size:]

In [ ]:
print(train_texts)

['package main\n\n/*\nAll structures here matches the manifest template.\nThey are kept here to provide example and contract between manifest and backend.\nPlease update / add / remove as needed.\n*/\n\ntype Connection struct {\n\tHostnameOrAddress string `json:"hostnameOrAddress"`\n\tUsername          string `json:"username"`\n\tPassword          string `json:"password"`\n\tPort              int    `json:"port"`\n}\n\ntype sslProfileType string\n\nconst (\n\tClientSSLProfile = sslProfileType("client")\n\tServerSSLProfile = sslProfileType("server")\n)\n\ntype SSLProfile struct {\n\tName      string         `json:"name"`\n\tType      sslProfileType `json:"type"`\n\tPartition string         `json:"partition"`\n}\n\ntype CertificateBundle struct {\n\tCertificate      []byte   `json:"certificate"`\n\tPrivateKey       []byte   `json:"privateKey"`\n\tCertificateChain [][]byte `json:"certificateChain"`\n}\n\ntype Keystore struct {\n\tPartition       string `json:"partition"`\n\tCertificateNam

In [ ]:
def textFile(dataText,name):
  all_text=""
  for text in dataText:
   # print(description)
    all_text=all_text+ text + '<|endoftext|> \n '
  with open(
                f'/content/sample_data/{name}.txt', 'w',
             encoding="utf-8") as f:  
    
              #print(test)
            # print(f"lol", f.write(' <|startoftext|> '+ '[DES]'+ description +'[TIT]' + title+'.' + " <|endoftext|> "))
              print(f"{all_text}", file=f)    

In [ ]:
textFile(train_texts,"train")
textFile(val_texts,"val")


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
# Convert the data to a dataset format
# train_dataset = TextDataset(train_encodings)
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/sample_data/train.txt',
    block_size=128)

val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/sample_data/val.txt',
    block_size=128)


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Set up the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
)

# Create a Trainer instance and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("/drive/MyDrive/fine-tuned-codegen-multi-350M")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4
  Number of trainable parameters = 354860152
/usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/fine-tuned-codegen-multi-350M
Configuration saved in /content/fine-tuned-codegen-multi-350M/config.json
Configuration saved in /content/fine-tuned-codegen-multi-350M/generation_config.json
Model weights saved in /content/fine-tuned-codegen-multi-350M/pytorch_model.bin
